In [ ]:
from mpcontribs.client import load_client

In [ ]:
client = load_client('your-api-key-here')

Get and update project info (see https://portal.mpcontribs.org/carrier_transport)

In [ ]:
project = 'carrier_transport'
client.projects.get_entry(pk=project, _fields=['_all']).result()

## specific contribution

In [ ]:
contrib = client.contributions.get_entry(pk='5ac0a333d4f1443de26744ab', _fields=['_all']).result()

### hierarchical data

In [ ]:
contrib['data']

### tabular data

In [ ]:
tid = contrib['tables'][0]['id']
client.tables.get_entry(pk=tid, _fields=['_all']).result()

## query by materials identifiers

In [ ]:
identifiers = ['mp-2715', 'mp-988', 'mp-9899']
client.contributions.get_entries(
    identifier__in=identifiers,
    _fields=['identifier', 'data.formula', 'data.<S>']
).result()['data']

## query by values

In [ ]:
# see https://portal.mpcontribs.org/<project> and
# https://api.mpcontribs.org/#/contributions/get_entries

limit = 20 # 200 is the limit of retrievable #contributions per page
fields = ['<S>', '<σ>', '<S²σ>'] # which data fields to retrieve
mask = [f'data.{field}' for field in fields]
mask += ['formula', 'identifier']
filters = {
    'formula__contains': 'Li3',
#     'data__<σ>__p__lt': 2e15,
#     'data__<σ>__n__lt': 2e15
}

contribs = client.contributions.get_entries(
    project=project, _fields=mask, _limit=limit, **filters
).result() # -> see pagination

print('found', contribs['total_count'], 'materials')
contribs['data'][0]

## pagination

In [ ]:
# TODO outdated
# only run this once you've optimized masks and filters to what you actually need (see above)
all_contribs = []
while 1:
    page = int(len(all_contribs) / per_page) + 1
    contribs = client.contributions.get_entries(
        projects=[project], filters=filters, mask=mask, per_page=per_page, page=page
    ).result()
    all_contribs.extend(contribs)
    if not contribs or len(contribs) < per_page:
        break

print('found', len(all_contribs), 'materials')
all_contribs[-1]